In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import time
import csv
from PY_OZNAL_UTILS.oznal_utils import OZNAL_UTIL


In [22]:

def loadCommentData(fromFS=False):
    if fromFS:
        df = pd.read_csv("data_end/comments.csv")
        df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
        df = df.drop(columns=['Unnamed: 0'])
        return df

    df = pd.read_csv('data_spark/comments.csv')
    df = df.drop(columns=['_c0','Unnamed: 0','body','id'])
    df = df.rename(columns={"created_utc": "date", "total_awards_received" : "awards"})
    df['score'] = df['score'].fillna(1)
    df['awards'] =  df['awards'].fillna(0)
    df = df.dropna()
    df = df.astype({'score': 'int32', 'awards':'int32'})
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d %H:%M:%S")
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df.to_csv("data_spark/comments_filtered.csv")

        # read saved dataset
    with open('data_spark/comments_filtered.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        # create new dataset
        with open('data_spark/comments_split.csv', mode='w') as newFile:
            newCsv = csv.writer(newFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL )
            newCsv.writerow(['symbols', 'score', 'date','awards','sentiment'])
            for row in csv_reader:
                symbols = row['symbols'].replace('[', '').replace(']', '').replace('\'', '').split(',')
                score = row['score']
                date = row['date']
                awards = row['awards']
                sentiment = row['sentiment']
                for symbol in symbols:
                    newCsv.writerow([symbol, score, date, awards,sentiment])

        ###########################################################

        # read created dataset
        df = pd.read_csv('data_spark/comments_split.csv')
        # cast types
        df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
        # merge date with symbol
        df['concatColumn'] = df.apply(lambda x : x['symbols'] + " " + str(x['date']), axis=1)
        df['count'] = 1
        df = df.groupby('concatColumn').sum()
        # unmerge date and symbol
        df['symbol'] = ""
        df['date'] = ""
        for i, value in enumerate(df.index):
            df['symbol'][i] = str(value).strip().split(" ")[0]
            df['date'][i] =  str(value).strip().split(" ")[1]
        df.index = range(0, len(df))
        #cast date
        df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
        # unique symbols
        uniqueSymbols = list(df['symbol'].unique()) 
        df = df.sort_values(by='date')
        start = df['date'].loc[1]
        end =  df['date'].iloc[-1]
        print(start,end)
        ###########################################################

        # stock_final = pd.DataFrame()
        # OZNAL = OZNAL_UTIL()
        # symbols = OZNAL.getStockSymbols()
        # for symbol in uniqueSymbols:
        #     if symbol in  ['BABY', 'LONG', 'GAME', 'ACAT','ADA','AMDA','ARCI', 'ARCI','BBRY', 'BNB']:
        #         continue
        #     try:
        #         x = OZNAL.downloadNASDAQHistoricalStockData(symbol,start,end)
        #         x['date'] = x.index
        #         result = pd.merge(df, x, on=["symbol","date"])
        #         stock_final = stock_final.append(result)
        #     except:
        #         pass

        df.to_csv("data_end/comments.csv")
        return df

In [23]:
def loadPostData(fromFS=False):

    if fromFS:
        df = pd.read_csv("data_end/posts.csv")
        df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
        df = df.drop(columns=['Unnamed: 0'])
        return df

    # load  data set from spark
    df = pd.read_csv('data_spark/posts.csv')
    # drop un used columns
    df = df.drop(columns=['_c0','Unnamed: 0','selftext','id', 'title'])
    # rename columns
    df = df.rename(columns={"created_utc": "date", "num_comments" : "awards" })
    # fill na
    df['score'] = df['score'].fillna(1)
    df['awards'] = df['awards'].fillna(0)
    df['upvote_ratio'] = df['upvote_ratio'].fillna(0)

    # cast columns
    df = df.astype({'score': 'float32', "awards" : "float32"})
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    # save daset
    df.to_csv("data_spark/posts_1.csv")

    ###########################################################

    # read saved dataset
    with open('data_spark/posts_1.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        # create new dataset
        with open('data_spark/post_1_new.csv', mode='w') as newFile:
            newCsv = csv.writer(newFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL )
            newCsv.writerow(['symbols', 'score', 'date','awards','upvote_ratio','sentiment'])
            for row in csv_reader:
                symbols = row['symbols'].replace('[', '').replace(']', '').replace('\'', '').split(',')
                score = row['score']
                date = row['date']
                awards = row['awards']
                upV = row['upvote_ratio']
                sentiment=row['sentiment']
                for symbol in symbols:
                    newCsv.writerow([symbol, score, date, awards,upV,sentiment])

    ###########################################################

    # read created dataset
    df = pd.read_csv('data_spark/post_1_new.csv')
    # cast types
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
    # merge date with symbol
    df['concatColumn'] = df.apply(lambda x : x['symbols'] + " " + str(x['date']), axis=1)
    df['count'] = 1
    df = df.groupby('concatColumn').sum()
    # unmerge date and symbol
    df['symbol'] = ""
    df['date'] = ""
    for i, value in enumerate(df.index):
        df['symbol'][i] = str(value).strip().split(" ")[0]
        df['date'][i] =  str(value).strip().split(" ")[1]
    df.index = range(0, len(df))
    #cast date
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
    # unique symbols
    uniqueSymbols = list(df['symbol'].unique()) 
    df = df.sort_values(by='date')
    start = df['date'].loc[1]
    end =  df['date'].iloc[-1]

    ###########################################################

    # stock_final = pd.DataFrame()
    # OZNAL = OZNAL_UTIL()
    # symbols = OZNAL.getStockSymbols()
    # for symbol in uniqueSymbols:
    #     if symbol in  ['BABY', 'LONG', 'GAME', 'ACAT','ADA','AMDA','ARCI', 'ARCI','BBRY', 'BNB']:
    #         continue
    #     try:
    #         x = OZNAL.downloadNASDAQHistoricalStockData(symbol,start,end)
    #         x['date'] = x.index
    #         result = pd.merge(df, x, on=["symbol","date"])
    #         stock_final = stock_final.append(result)
    #     except:
    #         pass

    df.to_csv("data_end/post.csv")
    return df


In [24]:
%%time
df = loadPostData(fromFS=False)


C:\Users\ramang\AppData\Roaming\Python\Python38\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)
<ipython-input-23-dd1f82abd941>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['symbol'][i] = str(value).strip().split(" ")[0]
<ipython-input-23-dd1f82abd941>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i] =  str(value).strip().split(" ")[1]
Wall time: 2min 38s


In [25]:
%%time
df2 = loadCommentData(fromFS=False)

C:\Users\ramang\AppData\Roaming\Python\Python38\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (0,2,3,5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)
<ipython-input-22-0e0017e435aa>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['symbol'][i] = str(value).strip().split(" ")[0]
<ipython-input-22-0e0017e435aa>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i] =  str(value).strip().split(" ")[1]
2020-12-09 00:00:00 2021-02-03 00:00:00
Wall time: 2min 44s


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97943 entries, 70442 to 63903
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   score         97943 non-null  float64       
 1   awards        97943 non-null  float64       
 2   upvote_ratio  97943 non-null  float64       
 3   sentiment     97943 non-null  float64       
 4   count         97943 non-null  int64         
 5   symbol        97943 non-null  object        
 6   date          97943 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 8.0+ MB


In [27]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77991 entries, 0 to 34161
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   score      77991 non-null  int64         
 1   awards     77991 non-null  int64         
 2   sentiment  77991 non-null  float64       
 3   count      77991 non-null  int64         
 4   symbol     77991 non-null  object        
 5   date       77991 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 6.2+ MB


In [28]:
df.head()

,score,awards,upvote_ratio,sentiment,count,symbol,date
70442,1.0,10.0,0.0,0.0000,1,NFLX,2019-01-01
87448,1.0,8.0,0.0,0.0000,1,TFSA,2019-01-01
5401,1.0,1.0,0.0,0.3802,1,CALM,2019-01-01
88873,3.0,89.0,0.0,1.0658,3,TSLA,2019-01-01
35333,1.0,10.0,0.0,0.8221,1,AMD,2019-01-01


In [29]:
result = pd.merge(df, df2, on=["symbol","date"], how="outer")
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163920 entries, 0 to 163919
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   score_x       116795 non-null  float64       
 1   awards_x      116795 non-null  float64       
 2   upvote_ratio  116795 non-null  float64       
 3   sentiment_x   116795 non-null  float64       
 4   count_x       116795 non-null  float64       
 5   symbol        163920 non-null  object        
 6   date          163920 non-null  datetime64[ns]
 7   score_y       91185 non-null   float64       
 8   awards_y      91185 non-null   float64       
 9   sentiment_y   91185 non-null   float64       
 10  count_y       91185 non-null   float64       
dtypes: datetime64[ns](1), float64(9), object(1)
memory usage: 15.0+ MB


In [30]:
result.head()

,score_x,awards_x,upvote_ratio,sentiment_x,count_x,symbol,date,score_y,awards_y,sentiment_y,count_y
0,1.0,10.0,0.0,0.0000,1.0,NFLX,2019-01-01,NaN,NaN,NaN,NaN
1,1.0,8.0,0.0,0.0000,1.0,TFSA,2019-01-01,NaN,NaN,NaN,NaN
2,1.0,1.0,0.0,0.3802,1.0,CALM,2019-01-01,NaN,NaN,NaN,NaN
3,3.0,89.0,0.0,1.0658,3.0,TSLA,2019-01-01,NaN,NaN,NaN,NaN
4,1.0,10.0,0.0,0.8221,1.0,AMD,2019-01-01,NaN,NaN,NaN,NaN


In [31]:
%%time

def superFuc(row):
    columnNames = ["score","awards","sentiment","count"]
    for columnName in columnNames:
        row[columnName] = row[columnName+"_x"] + row[columnName+"_y"]
    return row

result['score_x'] = result['score_x'].fillna(1)
result['awards_x'] =  result['awards_x'].fillna(0)
result['upvote_ratio'] =  result['upvote_ratio'].fillna(1)
result['score_y'] = result['score_y'].fillna(1)
result['awards_y'] =  result['awards_y'].fillna(0)
result['sentiment_y'] =  result['sentiment_y'].fillna(0)
result['sentiment_x'] =  result['sentiment_x'].fillna(0)
result['count_x'] =  result['count_x'].fillna(0)
result['count_y'] =  result['count_y'].fillna(0)
dfMerged = result.apply(lambda row: superFuc(row), axis=1)
dfMerged

Wall time: 4min 23s


,score_x,awards_x,upvote_ratio,sentiment_x,count_x,symbol,date,score_y,awards_y,sentiment_y,count_y,score,awards,sentiment,count
0,1.0,10.0,0.0,0.0000,1.0,NFLX,2019-01-01,1.0,0.0,0.0000,0.0,2.0,10.0,0.0000,1.0
1,1.0,8.0,0.0,0.0000,1.0,TFSA,2019-01-01,1.0,0.0,0.0000,0.0,2.0,8.0,0.0000,1.0
2,1.0,1.0,0.0,0.3802,1.0,CALM,2019-01-01,1.0,0.0,0.0000,0.0,2.0,1.0,0.3802,1.0
3,3.0,89.0,0.0,1.0658,3.0,TSLA,2019-01-01,1.0,0.0,0.0000,0.0,4.0,89.0,1.0658,3.0
4,1.0,10.0,0.0,0.8221,1.0,AMD,2019-01-01,1.0,0.0,0.0000,0.0,2.0,10.0,0.8221,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163915,1.0,0.0,1.0,0.0000,0.0,BK,2021-02-03,12.0,0.0,0.7876,2.0,13.0,0.0,0.7876,2.0
163916,1.0,0.0,1.0,0.0000,0.0,XOM,2021-02-03,6.0,0.0,0.1789,2.0,7.0,0.0,0.1789,2.0
163917,1.0,0.0,1.0,0.0000,0.0,TNDM,2021-02-03,2.0,0.0,0.3182,1.0,3.0,0.0,0.3182,1.0
163918,1.0,0.0,1.0,0.0000,0.0,WSO,2021-02-03,26.0,0.0,0.6137,1.0,27.0,0.0,0.6137,1.0


In [43]:
dfMerged = dfMerged.loc[:,~dfMerged.columns.str.contains('_y|_x', case=False)] 

1498

In [49]:
%%time
stock_final = pd.DataFrame()
uniqueSymbols = list(dfMerged['symbol'].unique())
start = dfMerged['date'].loc[1]
end =  dfMerged['date'].iloc[-1]
OZNAL = OZNAL_UTIL()
symbols = OZNAL.getStockSymbols()
for symbol in uniqueSymbols:
    try:
        x = OZNAL.downloadNASDAQHistoricalStockData(symbol,start,end)
        x['date'] = x.index
        result = pd.merge(dfMerged, x, on=["symbol","date"])
        aapl = yf.Ticker(symbol)
        result['float'] = aapl.info['floatShares']
        stock_final = stock_final.append(result)
    except:
        try:
            x = OZNAL.downloadNASDAQHistoricalStockData(symbol+"-USD",start,end)
            x['date'] = x.index
            result = pd.merge(dfMerged, x, on=["symbol","date"])
            result['float'] = 0
            stock_final = stock_final.append(result)
        except:
            pass
        pass
#stock_final.to_csv("data_end/final_2.csv")


ad:
- AMZA-USD: No data found, symbol may be delisted

1 Failed download:
- OLO: Data doesn't exist for startDate = 1546297200, endDate = 1612306800

1 Failed download:
- OLO-USD: No data found, symbol may be delisted

1 Failed download:
- BSA-USD: No data found, symbol may be delisted

1 Failed download:
- PGM-USD: No data found, symbol may be delisted

1 Failed download:
- PRS-USD: No data found, symbol may be delisted

1 Failed download:
- VIOO-USD: No data found, symbol may be delisted

1 Failed download:
- ELD-USD: No data found, symbol may be delisted

1 Failed download:
- IJT-USD: No data found, symbol may be delisted

1 Failed download:
- IYJ-USD: No data found, symbol may be delisted

1 Failed download:
- MRAC: Data doesn't exist for startDate = 1546297200, endDate = 1612306800

1 Failed download:
- MRAC-USD: No data found, symbol may be delisted

1 Failed download:
- FGM-USD: No data found, symbol may be delisted

1 Failed download:
- CLOVW-USD: No data found, symbol may be d

In [50]:
stock_final.to_csv("data_end/final_withFloat.csv")

In [51]:
stock_final

,upvote_ratio,symbol,date,score,awards,sentiment,count,Open,High,Low,Close,Adj Close,Volume,float
0,0.0,NFLX,2019-01-03,4.0,6.0,0.5423,3.0,270.200012,275.790009,264.429993,271.200012,271.200012,14969600.0,437053473
1,0.0,NFLX,2019-01-04,3.0,5.0,0.0000,2.0,281.880005,297.799988,278.540009,297.570007,297.570007,19330100.0,437053473
2,0.0,NFLX,2019-01-08,5.0,158.0,0.4571,4.0,319.980011,320.589996,308.010010,320.269989,320.269989,15359200.0,437053473
3,0.0,NFLX,2019-01-09,3.0,1.0,0.0000,2.0,317.709991,323.350006,313.500000,319.959991,319.959991,13343200.0,437053473
4,0.0,NFLX,2019-01-10,2.0,0.0,0.0000,1.0,314.570007,325.369995,312.500000,324.660004,324.660004,13472500.0,437053473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1.0,AOD,2021-02-02,2.0,0.0,0.8574,1.0,9.090000,9.150000,9.080000,9.100000,8.988989,313700.0,None
0,1.0,ARNA,2021-02-02,3.0,0.0,0.8677,1.0,77.879997,80.010002,77.519997,79.339996,79.339996,577000.0,55996587
0,1.0,MAV,2021-02-02,2.0,0.0,0.9042,1.0,11.770000,11.820000,11.730000,11.750000,11.595755,64900.0,None
0,1.0,TSIA,2021-02-02,-3.0,0.0,0.8283,2.0,15.980000,17.250000,15.850000,17.090000,17.090000,4065100.0,33000000


In [43]:
# dfMerged1 = dfMerged
# start = dfMerged['date'].loc[1]
# end =  dfMerged['date'].iloc[-1]
# OZNAL = OZNAL_UTIL()
# uniqueSymbols = OZNAL.getStockSymbols()
# x = yf.download(uniqueSymbols,start,end)
# symbols = uniqueSymbols
# columns = list(set([x[0] for x in x.columns]))
# x.columns = x.columns.map('_'.join)

In [141]:
# stock_final = pd.DataFrame()

# for s in symbols[:-1]:
#     columnsX = [x+"_"+s for x in columns]
#     if str(x[columnsX].iloc[0][0]) == "nan":
#         continue
#     else:
#         stock = x[columnsX]
#         stock.columns = stock.columns.map(lambda x : x.split("_")[0])
#         stock['symbol'] = s
#         stock['date'] = stock.index
#         result = pd.merge(dfMerged1[dfMerged1['symbol'] == s], stock, on=["symbol","date"])
#         stock_final = stock_final.append(result)

[*********************100%***********************]  4495 of 4495 completed

591 Failed downloads:
- CAHCW: No data found for this date range, symbol may be delisted
- LMACW: Data doesn't exist for startDate = 1546470000, endDate = 1613430000
- METXW: No data found for this date range, symbol may be delisted
- PPTA: Data doesn't exist for startDate = 1546470000, endDate = 1613430000
- TIOAU: Data doesn't exist for startDate = 1546470000, endDate = 1613430000
- TCACW: No data found for this date range, symbol may be delisted
- PFDR: No data found for this date range, symbol may be delisted
- DHBCU: Data doesn't exist for startDate = 1546470000, endDate = 1613430000
- TMKRW: No data found for this date range, symbol may be delisted
- FMIVU: Data doesn't exist for startDate = 1546470000, endDate = 1613430000
- CMIIU: Data doesn't exist for startDate = 1546470000, endDate = 1613430000
- BCAC: Data doesn't exist for startDate = 1546470000, endDate = 1613430000
- LBPS: Data doesn't exist for 

<ipython-input-141-a46aceda8382>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['symbol'] = s
<ipython-input-141-a46aceda8382>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['date'] = stock.index


,score,awards,upvote_ratio,sentiment,symbol,date,Volume,High,Adj Close,Open,Low,Close


In [57]:
# crypto = [
#             'BTC',
#             'ETH',
#             'XMR',
#             'XRP',
#             'LTC',
#             'BNB',
#             'USDT',
#             'DOT',
#             'ADA',
#             'UNI',
#             'LTC',
#             'XLM',
#             'LINK',
#             'BCH',
#             'THETA',
#             'FIL',
#             'USDC',
#             'DOGE',
#             'WBTC',
#             'VET',
#             'SOL',
#             'EOS',
#             'MIOTA',
#             'LUNA',
#             'BTT'
#             'XTZ',
#             'ATOM',
#             'AVAX',
#             'ALGO'
#         ]
# cryptoSymbols = [ c+"-USD" for c in crypto]

[*********************100%***********************]  27 of 27 completed

8 Failed downloads:
- SOL-USD: No data found, symbol may be delisted
- LUNA-USD: No data found, symbol may be delisted
- WBTC-USD: No data found, symbol may be delisted
- ATOM-USD: No data found, symbol may be delisted
- BTTXTZ-USD: No data found, symbol may be delisted
- FIL-USD: No data found, symbol may be delisted
- UNI-USD: No data found, symbol may be delisted
- DOT-USD: No data found, symbol may be delisted


In [126]:
# start = dfMerged['date'].loc[1]
# end =  dfMerged['date'].iloc[-1]
# cryptoFinance = yf.download(cryptoSymbols,start,end)
# columns = list(set([x[0] for x in cryptoFinance.columns]))
# cryptoFinance.columns = cryptoFinance.columns.map('_'.join)
# for s in cryptoSymbols:
#     columnsX = [x+"_"+s for x in columns]
#     if str(cryptoFinance[columnsX].iloc[0][0]) == "nan":
#         continue
#     else:
#         stock = cryptoFinance[columnsX]
#         stock.columns = stock.columns.map(lambda x : x.split("_")[0])
#         stock['symbol'] = s.split("-")[0]
#         stock['date'] = stock.index

#         result = pd.merge(dfMerged1[dfMerged1['symbol'] == s.split("-")[0]], stock, on=["symbol","date"])
#         stock_final = stock_final.append(result)

[*********************100%***********************]  27 of 27 completed

8 Failed downloads:
- SOL-USD: No data found, symbol may be delisted
- LUNA-USD: No data found, symbol may be delisted
- WBTC-USD: No data found, symbol may be delisted
- ATOM-USD: No data found, symbol may be delisted
- BTTXTZ-USD: No data found, symbol may be delisted
- FIL-USD: No data found, symbol may be delisted
- UNI-USD: No data found, symbol may be delisted
- DOT-USD: No data found, symbol may be delisted


<ipython-input-126-921faf9c0b90>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['symbol'] = s.split("-")[0]
<ipython-input-126-921faf9c0b90>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['date'] = stock.index


In [127]:
# stock_final[stock_final['symbol'] == 'BTC']

,score,awards,upvote_ratio,sentiment,symbol,date,Volume,High,Adj Close,Open,Low,Close
0,1.0,0.0,0.0,0.0772,BTC,2019-01-05,5.137610e+09,3904.903076,3845.194580,3851.973877,3836.900146,3845.194580
1,1.0,0.0,0.0,0.6705,BTC,2019-01-06,5.597027e+09,4093.297363,4076.632568,3836.519043,3826.513184,4076.632568
2,1.0,0.0,0.0,0.0000,BTC,2019-01-13,4.681302e+09,3674.760010,3552.953125,3658.868164,3544.927246,3552.953125
3,3.0,1.0,0.0,0.4404,BTC,2019-01-14,5.651384e+09,3727.836182,3706.052246,3557.311035,3552.285156,3706.052246
4,1.0,0.0,0.0,-0.2960,BTC,2019-01-16,5.394457e+09,3685.777100,3655.006836,3631.509766,3624.673340,3655.006836
...,...,...,...,...,...,...,...,...,...,...,...,...
520,1.0,0.0,0.0,0.2023,BTC,2021-02-09,9.180985e+10,48003.722656,46481.105469,46184.992188,45166.960938,46481.105469
521,2.0,0.0,0.0,0.0000,BTC,2021-02-10,8.730109e+10,47145.566406,44918.183594,46469.761719,43881.152344,44918.183594
522,1.0,2.0,0.0,0.0000,BTC,2021-02-12,7.655504e+10,48745.734375,47504.851562,47877.035156,46424.976562,47504.851562
523,1.0,2.0,0.0,0.0000,BTC,2021-02-13,7.025046e+10,48047.746094,47105.515625,47491.203125,46392.281250,47105.515625
